In [39]:
### NOT NEEDED TO RUN THIS ONE ###

### Pad images to a common chosen size (180, 150)

import os
from os import listdir

from PIL import Image
from PIL import ImageOps
 
# get the path/directory
folder_dir = "C:\\Users\\Hector\Desktop\\Ivan_dissertation\\SHADOW\\"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".jpg")):

        image = Image.open('C:\\Users\\Hector\Desktop\\Ivan_dissertation\\SHADOW\\'+images).convert("RGB")
        ImageOps.pad(image,(180,150)).save('C:\\Users\\Hector\Desktop\\Ivan_dissertation\\SHADOW_r\\'+images)

In [67]:
# Import Deep Learning Libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [64]:
# Create train and validation datasets

# Replace this variable for your directory of the downloaded pictures - in this case, the folder Dataset contains the 15 folders
path = 'C:\\Users\\Hector\Desktop\\Ivan_dissertation\\Dataset\\'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="training",
  seed=1337,
  image_size=(180, 150),
  batch_size=32)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="validation",
  seed=1337,
  image_size=(180, 150),
  batch_size=32)

Found 1836 files belonging to 15 classes.
Using 1469 files for training.
Found 1836 files belonging to 15 classes.
Using 367 files for validation.


In [65]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=15)
#keras.utils.plot_model(model, show_shapes=True)

In [66]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/25
46/46 [==============================] - 229s 5s/step - loss: 1.2023 - accuracy: 0.5936 - val_loss: 2.5357 - val_accuracy: 0.2643


C:\Users\Hector\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/25
46/46 [==============================] - 258s 6s/step - loss: 0.7814 - accuracy: 0.7318 - val_loss: 2.3290 - val_accuracy: 0.2071
Epoch 3/25
46/46 [==============================] - 280s 6s/step - loss: 0.6662 - accuracy: 0.7849 - val_loss: 2.2546 - val_accuracy: 0.2071
Epoch 4/25
46/46 [==============================] - 274s 6s/step - loss: 0.6003 - accuracy: 0.7965 - val_loss: 2.3005 - val_accuracy: 0.2071
Epoch 5/25
46/46 [==============================] - 262s 6s/step - loss: 0.5285 - accuracy: 0.8121 - val_loss: 2.4482 - val_accuracy: 0.2071
Epoch 6/25
46/46 [==============================] - 263s 6s/step - loss: 0.5069 - accuracy: 0.8319 - val_loss: 2.2714 - val_accuracy: 0.2452
Epoch 7/25
46/46 [==============================] - 263s 6s/step - loss: 0.5224 - accuracy: 0.8264 - val_loss: 2.7471 - val_accuracy: 0.2071
Epoch 8/25
46/46 [==============================] - 263s 6s/step - loss: 0.4434 - accuracy: 0.8468 - val_loss: 2.5333 - val_accuracy: 0.2044
Epoch 9/25
46

KeyboardInterrupt: 